# 💬 Agent와 Multi-turn 대화하기

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 12px; color: white; margin-bottom: 20px;">

### 🔄 지속적인 대화 구현

에이전트와 여러 차례에 걸친 대화를 진행하는 방법을 배웁니다.
</div>

<div style="background: #e3f2fd; border-left: 4px solid #2196f3; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 💡 대화 상태 관리의 핵심

에이전트는 기본적으로 **상태를 저장하지 않습니다**. 각 요청은 독립적으로 처리됩니다.

**Multi-turn 대화를 위한 해결책:**
- 📦 **AgentThread 객체** 생성
- 🔄 대화 상태를 스레드에 저장
- 📤 에이전트 실행 시 스레드 전달

**스레드 생성 방법:**
```python
thread = agent.get_new_thread()
```

</div>

---

## 1️⃣ 에이전트 및 스레드 생성

<div style="background: #e8f5e9; border-left: 4px solid #4caf50; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 🚀 시작 준비

**단계:**
1. 🤖 에이전트 인스턴스 생성
2. 🧵 새로운 스레드 생성 (`get_new_thread()`)

</div>

In [ ]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# 새로운 에이전트 생성
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="당신은 여행 장소를 추천해주는, 친절한 관광 어시스턴트입니다. 😄🎭",
    name="TravelAssistant",
)

# 새로운 쓰레드 생성
thread = agent.get_new_thread()


---

## 2️⃣ 다중 턴 대화 실행

<div style="background: #f3e5f5; border-left: 4px solid #9c27b0; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 🔗 연속된 대화 진행

동일한 `thread` 객체를 사용하여 여러 번 에이전트를 실행하면 이전 대화 내용을 기억합니다.

**대화 흐름:**
1. 💬 첫 번째 질문: 여행지 요청
2. 🔄 두 번째 질문: 이전 추천 여행지 중 딱 한 군데만 재추천 (스레드 컨텍스트 활용)

**핵심:** 두 번째 질문에서 "그 여행지"라고 해도 에이전트가 이해합니다!

</div>

In [ ]:
async def main():
    result1 = await agent.run(
        "서울에서 반드시 가봐야 하는 관광 명소를 5군데만 추천해줘.", 
        thread=thread)
    print("결과 1: " + result1.text)
    print("=" * 70)
    result2 = await agent.run(
        "그 여행지들 중 딱 하나만 골라준다면? 😄", 
        thread=thread)
    print("결과 2: " + result2.text)

await main()

<div style="background: #fff3e0; border-left: 4px solid #ff9800; padding: 15px; border-radius: 8px; margin: 15px 0;">

### ⚡ 중요: 대화 기록 저장 방식

대화 기록의 저장 위치는 사용하는 서비스 유형에 따라 달라집니다:

| 서비스 유형 | 저장 위치 | 전송 방식 |
|------------|----------|----------|
| 💬 **Chat Completion** | `AgentThread` 객체 (로컬) | 전체 대화 기록을 매번 전송 |
| ☁️ **Azure AI Agent** | Azure AI Agent 서비스 (클라우드) | 대화 참조 ID만 전송 |

**이 예제에서는:**
- Chat Completion 서비스 사용
- 대화 기록이 AgentThread에 저장
- 각 호출 시 전체 대화 기록 전송

</div>

---

## 3️⃣ 멀티 스레드: 여러 독립적인 대화 관리

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 20px; border-radius: 12px; color: white; margin: 15px 0;">

### 🎭 하나의 에이전트, 여러 대화

동일한 에이전트로 완전히 독립적인 여러 대화를 동시에 진행할 수 있습니다!

</div>

<div style="background: #e8f5e9; border-left: 4px solid #4caf50; padding: 15px; border-radius: 8px; margin: 15px 0;">

**시나리오:**
- 🧵 **Thread 1**: 한국 추천 여행지에 대한 대화
- 🧵 **Thread 2**: 미국 추천 여행지에 대한 대화

**특징:**
- ✅ 각 스레드는 완전히 독립적
- ✅ 에이전트는 상태 비저장 (stateless)
- ✅ 스레드별로 별도의 컨텍스트 유지

**실행 흐름:**
1. 📝 Thread 1에서 한국 추천 여행지 요청
2. 📝 Thread 2에서 미국 추천 여행지 요청
3. 🔄 Thread 1에서 한국 추천 여행지 중 한 곳 재추천 (첫 번째 대화 기억)
4. 🔄 Thread 2에서 미국 추천 여행지 중 한 곳 재추천 (두 번째 대화 기억)

</div>

In [ ]:
async def main():
    thread1 = agent.get_new_thread()
    thread2 = agent.get_new_thread()

    result1 = await agent.run("한국에서 반드시 가봐야 하는 관광 명소를 3군데만 추천해줘", thread=thread1)
    print("결과 1: " + result1.text)
    print("=" * 70)
    result2 = await agent.run("미국에서 반드시 가봐야 하는 관광 명소를 3군데만 추천해줘", thread=thread2)
    print("결과 2: " + result2.text)
    print("=" * 70)
    result3 = await agent.run("그 여행지들 중 딱 하나만 골라준다면? 😄", thread=thread1)
    print("결과 1-1: " + result3.text)
    print("=" * 70)
    result4 = await agent.run("그 여행지들 중 딱 하나만 골라준다면? 😄", thread=thread2)
    print("결과 2-1: " + result4.text)

await main()